# Shot Data
Amanda Kuznecov (anr431)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import poisson
import random
import scipy.stats as st
from scipy.special import logit, expit
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import brier_score_loss
from tqdm import tqdm
pd.set_option('display.max_columns', None)
import itertools

In [2]:
#read in data
df = pd.read_csv('soccer18_full.csv', index_col = False, 
                 parse_dates = ['Date'])

#sort by date to update ratings chronologically
df = df.sort_values('Date')

#rename columns
df = df.rename({'HomeTeam': 'Team_Home','AwayTeam': 'Team_Away',
                'FTHG': 'FTG_Home','FTAG': 'FTG_Away',
                'HTHG': 'HTG_Home','HTAG': 'HTG_Away', 
                'HS': 'S_Home', 'AS': 'S_Away', 'HST': 'ST_Home', 
               'AST': 'ST_Away','pH':'p_Home','pA': 'p_Away','home_xG': 'xG_Home',
               'away_xG': 'xG_Away','home_team_id':'team_id_Home','away_team_id':'team_id_Away'}, axis=1)

#read in shots data
df_shots = pd.read_csv('soccer18_shots.csv', index_col = False)

df_shots = df_shots.rename({'player':'player_shot'}, axis=1)

## Question 1
### Part a

In [3]:
#determine game without shot data
list(set(df.game_id.unique())-set(df_shots.game_id.unique()))

[4238]

In [4]:
df.loc[df.game_id == 4238]

,Div,Date,Y,Team_Home,Team_Away,FTG_Home,FTG_Away,HTG_Home,HTG_Away,S_Home,S_Away,ST_Home,ST_Away,p_Home,pD,p_Away,xG_Home,xG_Away,team_id_Home,team_id_Away,game_id,pOver,pUnder
9110,Ligue_1,2017-04-16,16,Bastia,Lyon,0,3,0,1,11,15,4,7,0.161632,0.221509,0.61686,0.0,3.0,162,178,4238,0.600484,0.399516


In [5]:
df = df[df.game_id != 4238]

After a quick Google search, it appears that this game was actually never finished! Bastia fans stormed the field and attacked the Lyon players, who then refused to continue play. The game was abandoned at half-time and since the game was never completed, the match statistics were not recorded as part of the shot data because all data stored in there is from completed matches.

### Part b

In [6]:
#keep relevant information for player xG
df_player = df_shots.loc[:,['shot_id','game_id','xG','player_assisted','player_shot']]

#df to store all players in single column
df_long = pd.wide_to_long(df_player, ['player'], i = ['shot_id','game_id'], 
                          j = 'play', sep = '_', suffix = r'\w+')

df_long = df_long.reset_index().sort_values(['shot_id','game_id']) 

#remove rows where no player assisted
df_long = df_long.dropna()

#merge df with season num
df_long = df_long.merge(df[['game_id','Y']], on = 'game_id', how = 'inner')

In [7]:
#group by player and season
group_player = df_long.groupby(['player','Y'])

#summary df of total xG per season per player
summary = group_player.agg(
    Total_xG = pd.NamedAgg(column = 'xG',aggfunc = 'sum')
    )

summary = summary.reset_index().sort_values(['player','Y']) 

#get top 10 results
summary = summary.sort_values('Total_xG',ascending = False)
summary[:10]

,player,Y,Total_xG
6301,Lionel Messi,14,53.503719
2049,Cristiano Ronaldo,14,52.909403
6562,Luis Suárez,15,49.109693
8085,Neymar,15,46.124972
2050,Cristiano Ronaldo,15,44.195410
6304,Lionel Messi,17,44.046686
6302,Lionel Messi,15,42.973189
9198,Robert Lewandowski,18,42.756660
6305,Lionel Messi,18,41.332334
6303,Lionel Messi,16,40.840305


### Part c

It would be biased to measure player ability solely by expected goals because xG will automatically favour players who get more scoring chances/opportunities. Some players have more playing time so it is likely they will have more expected goals. Also we would not be able to compare all players fairly because it is more likely that forwards and midfielders will have scoring chances, whereas defencemen will have very few to no expected goals, and goalies will have none.

### Part d

In [10]:
#column for header goals
df_shots['hg'] = 1*((df_shots.result == 'Goal') & (df_shots.shot_type == 'Head'))

In [11]:
#group each team in each game
group_gt = df_shots.groupby(['game_id','team_id'])

In [12]:
#summary df of total xG per season per player
new_cols = group_gt.agg(
    OG = pd.NamedAgg(column = 'result', aggfunc = lambda x: (x == 'OwnGoal').sum()),
    SP = pd.NamedAgg(column = 'result', aggfunc = lambda x: (x == 'ShotOnPost').sum()),
    HG = pd.NamedAgg(column = 'hg', aggfunc = 'sum')
    )
new_cols = new_cols.reset_index().sort_values(['game_id','team_id']) 

In [13]:
#get table containing just team/game indices corresponding with home/away
df_sm = df[['game_id','team_id_Home','team_id_Away']]

#pivot table to get all teams in one column and differentiated by home/away
df_long = pd.wide_to_long(df_sm, ['team_id'], i = ['game_id'], 
                          j = 'isHome', sep = '_', suffix = r'\w+')

df_long = df_long.reset_index().sort_values(['game_id']) 

#merge on teams in each game
df_long = df_long.merge(new_cols, left_on = ['game_id','team_id'],right_on = ['game_id','team_id'], how = 'outer')

In [14]:
#pivot to have all data for a particular game on one row
df_pivot = df_long.pivot(index='game_id', columns = 'isHome',values = ['OG','SP','HG'])

#rename columns and drop hierarchical level
df_pivot.columns = pd.MultiIndex.from_tuples([
    ('OG', 'OG_Away'), ('OG', 'OG_Home'), ('SP', 'SP_Away'),('SP', 'SP_Home'), ('HG','HG_Away'),('HG','HG_Home')
])

#remove top level of hierarchical column naming
df_pivot = df_pivot.droplevel(0,axis=1)

#merge soccer_full df with new shot data
df = df.merge(df_pivot, on = ['game_id'])

#fill nan values with 0
df = df.fillna(0)

In [15]:
df.loc[df.OG_Away +df.OG_Home == (df.OG_Away + df.OG_Home).max()][['Team_Home','Team_Away','OG_Home','OG_Away','Date','Div']]

,Team_Home,Team_Away,OG_Home,OG_Away,Date,Div
372,Southampton,Sunderland,0.0,3.0,2014-10-18,EPL
395,QPR,Liverpool,2.0,1.0,2014-10-19,EPL
1618,Empoli,Napoli,1.0,2.0,2015-04-30,Serie_A


In [17]:
df.loc[df.SP_Away +df.SP_Home == (df.SP_Away + df.SP_Home).max()][['Team_Home','Team_Away','SP_Home','SP_Away','Date','Div']]

,Team_Home,Team_Away,SP_Home,SP_Away,Date,Div
203,Sociedad,Almeria,4.0,1.0,2014-09-21,La_Liga
2795,Getafe,Espanol,4.0,1.0,2016-01-17,La_Liga
5409,West Ham,Liverpool,2.0,3.0,2017-05-14,EPL
6313,Barcelona,La Coruna,5.0,0.0,2017-12-17,La_Liga
8148,Hoffenheim,Mainz,4.0,1.0,2018-12-23,Bundesliga


In [20]:
df.loc[df.HG_Away +df.HG_Home == (df.HG_Away + df.HG_Home).max()][['Team_Home','Team_Away','HG_Home','HG_Away','Date','Div']]

,Team_Home,Team_Away,HG_Home,HG_Away,Date,Div
4080,Cagliari,Fiorentina,3.0,2.0,2016-10-23,Serie_A


### Part e

In [22]:
#calculate goal differentials for home and away teams of each match
df.loc[:,'GD_Home'] = df.loc[:,'FTG_Home']-df.loc[:,'FTG_Away']
df.loc[:,'GD_Away'] = df.loc[:,'FTG_Away']-df.loc[:,'FTG_Home']

#calculate expected goal differentials for home and away teams of each match
df.loc[:,'xGD_Home'] = df.loc[:,'xG_Home']-df.loc[:,'xG_Away']
df.loc[:,'xGD_Away'] = df.loc[:,'xG_Away']-df.loc[:,'xG_Home']

#calculate own goal differentials for home and away teams of each match
df.loc[:,'OGD_Home'] = df.loc[:,'OG_Home']-df.loc[:,'OG_Away']
df.loc[:,'OGD_Away'] = df.loc[:,'OG_Away']-df.loc[:,'OG_Home']

#calculate shots on post differentials for home and away teams of each match
df.loc[:,'SPD_Home'] = df.loc[:,'SP_Home']-df.loc[:,'SP_Away']
df.loc[:,'SPD_Away'] = df.loc[:,'SP_Away']-df.loc[:,'SP_Home']

#calculate header goal differentials for home and away teams of each match
df.loc[:,'HGD_Home'] = df.loc[:,'HG_Home']-df.loc[:,'HG_Away']
df.loc[:,'HGD_Away'] = df.loc[:,'HG_Away']-df.loc[:,'HG_Home']

In [23]:
#keep columns to compute num games and owngoal diff per team per season
df_g = df[['Date','game_id','Y','Team_Home','Team_Away','GD_Home','GD_Away','xGD_Home','xGD_Away',
           'OGD_Home','OGD_Away','SPD_Home','SPD_Away','HGD_Home','HGD_Away']].copy()

#pivot table to get team from each game on separate row
df_long = pd.wide_to_long(df_g, ['Team','GD','xGD','OGD','SPD','HGD'], i = ['game_id','Y'], 
                          j = 'isHome', sep = '_', suffix = r'\w+')

#reindex to get rid of multi-index
df_long = df_long.reset_index().sort_values(['Date','game_id','Y']) 


In [24]:
#cumulative sum of each feat per team for all preceding games in each season
#use Bayesian prior of 0
df_long = df_long.assign(GDcum = df_long.groupby(['Team','Y'])['GD'].transform(lambda x: x.cumsum().shift(1, fill_value = 0)),
                         xGDcum = df_long.groupby(['Team','Y'])['xGD'].transform(lambda x: x.cumsum().shift(1, fill_value = 0)),
                         OGDcum = df_long.groupby(['Team','Y'])['OGD'].transform(lambda x: x.cumsum().shift(1, fill_value = 0)),
                         SPDcum = df_long.groupby(['Team','Y'])['SPD'].transform(lambda x: x.cumsum().shift(1, fill_value = 0)),
                         HGDcum = df_long.groupby(['Team','Y'])['HGD'].transform(lambda x: x.cumsum().shift(1, fill_value = 0)),
                        )

#number of games previously played in the season per team
df_long.loc[:,'Num_Games'] = df_long.groupby(['Team','Y']).cumcount()

In [26]:
#pivot to have all data for a particular game on one row
df_pivot = df_long.pivot(index='game_id', columns = 'isHome',values = ['Num_Games','GDcum','xGDcum','OGDcum','SPDcum','HGDcum'])

In [27]:
#rename columns and drop hierarchical level
df_pivot.columns = pd.MultiIndex.from_tuples([
    ('Num_Games', 'Num_Games_Away'), ('Num_Games', 'Num_Games_Home'), 
    ('GDcum', 'GDcum_Away'),('GDcum', 'GDcum_Home'),
    ('xGDcum', 'xGDcum_Away'),('xGDcum', 'xGDcum_Home'),
    ('OGDcum', 'OGDcum_Away'),('OGDcum', 'OGDcum_Home'),
    ('SPDcum', 'SPDcum_Away'),('SPDcum', 'SPDcum_Home'),
    ('HGDcum', 'HGDcum_Away'),('HGDcum', 'HGDcum_Home'),
])

#remove top level of hierarchical column naming
df_pivot = df_pivot.droplevel(0,axis=1)

#merge soccer_full df with new shot data
df = df.merge(df_pivot, on = ['game_id'])

In [28]:
#weight of num games
nGames = 5

In [29]:
#create var for diff (Home-Away) of avg diffs using weight of nGames
df.loc[:,'goalVar'] = df.GDcum_Home/(df.Num_Games_Home + nGames) - df.GDcum_Away/(df.Num_Games_Away + nGames)
df.loc[:,'xgVar'] = df.xGDcum_Home/(df.Num_Games_Home + nGames) - df.xGDcum_Away/(df.Num_Games_Away + nGames)
df.loc[:,'ownGoalVar'] = df.OGDcum_Home/(df.Num_Games_Home + nGames) - df.OGDcum_Away/(df.Num_Games_Away + nGames)
df.loc[:,'shotonPostVar'] = df.SPDcum_Home/(df.Num_Games_Home + nGames) - df.SPDcum_Away/(df.Num_Games_Away + nGames)
df.loc[:,'headGoalVar'] = df.HGDcum_Home/(df.Num_Games_Home + nGames) - df.HGDcum_Away/(df.Num_Games_Away + nGames)

In [30]:
mod_1 = smf.ols('logit(p_Home)~ownGoalVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()

In [31]:
mod_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     277.4
Date:                Fri, 19 Mar 2021   Prob (F-statistic):           5.38e-61
Time:                        14:45:36   Log-Likelihood:                -8342.2
No. Observations:                6321   AIC:                         1.669e+04
Df Residuals:                    6319   BIC:                         1.670e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2238      0.011    -19.644      0.000      -0.246      -0.201
ownGoalVar    -2.5428      0.153    -16.656      0.000      -2.842      -2.243
==============================================================================
Omnibus:                       44.160   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               61.529
Skew:                          -0.079   Prob(JB):                     4.36e-14
Kurtosis:                       3.457   Cond. No.                         13.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
mod_2 = smf.ols('logit(p_Home)~ownGoalVar+goalVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()

In [33]:
mod_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                 1.123e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:54:22   Log-Likelihood:                -3686.5
No. Observations:                6321   AIC:                             7379.
Df Residuals:                    6318   BIC:                             7399.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2072      0.005    -37.969      0.000      -0.218      -0.196
ownGoalVar     0.3478      0.076      4.592      0.000       0.199       0.496
goalVar        0.9648      0.007    145.758      0.000       0.952       0.978
==============================================================================
Omnibus:                       76.282   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              131.654
Skew:                          -0.049   Prob(JB):                     2.58e-29
Kurtosis:                       3.700   Cond. No.                         13.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
mod_3 = smf.ols('logit(p_Home)~ownGoalVar+xgVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()

In [35]:
mod_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                 1.509e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        14:55:23   Log-Likelihood:                -2935.0
No. Observations:                6321   AIC:                             5876.
Df Residuals:                    6318   BIC:                             5896.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2044      0.005    -42.184      0.000      -0.214      -0.195
ownGoalVar    -0.1177      0.066     -1.771      0.077      -0.248       0.013
xgVar          1.2263      0.007    169.248      0.000       1.212       1.240
==============================================================================
Omnibus:                      106.763   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              208.030
Skew:                          -0.054   Prob(JB):                     6.71e-46
Kurtosis:                       3.882   Cond. No.                         13.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Part f

In [36]:
mod_1 = smf.ols('logit(p_Home)~shotonPostVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     2298.
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:01:41   Log-Likelihood:                -7497.9
No. Observations:                6321   AIC:                         1.500e+04
Df Residuals:                    6319   BIC:                         1.501e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2167      0.010    -21.737      0.000      -0.236      -0.197
shotonPostVar     2.2059      0.046     47.933      0.000       2.116       2.296
==============================================================================
Omnibus:                       21.274   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.939
Skew:                          -0.061   Prob(JB):                     2.33e-06
Kurtosis:                       3.289   Cond. No.                         4.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
mod_2 = smf.ols('logit(p_Home)~shotonPostVar+goalVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                 1.235e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:01:56   Log-Likelihood:                -3449.8
No. Observations:                6321   AIC:                             6906.
Df Residuals:                    6318   BIC:                             6926.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2060      0.005    -39.202      0.000      -0.216      -0.196
shotonPostVar     0.6177      0.027     22.676      0.000       0.564       0.671
goalVar           0.8855      0.007    128.159      0.000       0.872       0.899
==============================================================================
Omnibus:                       68.239   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.718
Skew:                          -0.026   Prob(JB):                     7.45e-26
Kurtosis:                       3.661   Cond. No.                         5.22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
mod_3 = smf.ols('logit(p_Home)~shotonPostVar+xgVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.831
Method:                 Least Squares   F-statistic:                 1.555e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:03:13   Log-Likelihood:                -2856.7
No. Observations:                6321   AIC:                             5719.
Df Residuals:                    6318   BIC:                             5740.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -0.2040      0.005    -42.633      0.000      -0.213      -0.195
shotonPostVar     0.3255      0.026     12.717      0.000       0.275       0.376
xgVar             1.1770      0.008    145.325      0.000       1.161       1.193
==============================================================================
Omnibus:                       97.027   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              183.664
Skew:                          -0.044   Prob(JB):                     1.31e-40
Kurtosis:                       3.831   Cond. No.                         5.43
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Part g

In [39]:
mod_1 = smf.ols('logit(p_Home)~headGoalVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     1432.
Date:                Fri, 19 Mar 2021   Prob (F-statistic):          1.21e-282
Time:                        15:05:07   Log-Likelihood:                -7832.5
No. Observations:                6321   AIC:                         1.567e+04
Df Residuals:                    6319   BIC:                         1.568e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.2182      0.011    -20.762      0.000      -0.239      -0.198
headGoalVar     1.9639      0.052     37.842      0.000       1.862       2.066
==============================================================================
Omnibus:                       44.719   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.586
Skew:                          -0.093   Prob(JB):                     6.98e-14
Kurtosis:                       3.442   Cond. No.                         4.94
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
mod_2 = smf.ols('logit(p_Home)~headGoalVar+goalVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.780
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                 1.120e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:05:22   Log-Likelihood:                -3692.1
No. Observations:                6321   AIC:                             7390.
Df Residuals:                    6318   BIC:                             7411.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.2072      0.005    -37.947      0.000      -0.218      -0.197
headGoalVar    -0.0982      0.031     -3.143      0.002      -0.159      -0.037
goalVar         0.9686      0.007    130.760      0.000       0.954       0.983
==============================================================================
Omnibus:                       68.560   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.888
Skew:                          -0.033   Prob(JB):                     6.84e-26
Kurtosis:                       3.660   Cond. No.                         5.77
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
mod_3 = smf.ols('logit(p_Home)~headGoalVar+xgVar',data = df[(df.Num_Games_Home + df.Num_Games_Away > 9) &(df.Y <18)]).fit()
mod_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          logit(p_Home)   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                 1.519e+04
Date:                Fri, 19 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:05:46   Log-Likelihood:                -2917.6
No. Observations:                6321   AIC:                             5841.
Df Residuals:                    6318   BIC:                             5862.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.2042      0.005    -42.257      0.000      -0.214      -0.195
headGoalVar     0.1637      0.027      6.160      0.000       0.112       0.216
xgVar           1.2077      0.008    153.625      0.000       1.192       1.223
==============================================================================
Omnibus:                      110.608   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              217.421
Skew:                          -0.060   Prob(JB):                     6.13e-48
Kurtosis:                       3.901   Cond. No.                         5.55
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""